In [2]:
from sage.crypto.sbox import SBox
from sage.crypto.boolean_function import BooleanFunction

S = [0x7,0x4,0xA,0x9,0x1,0xF,0xB,0x0,0xC,0x3,0x2,0x6,0x8,0xE,0xD,0x5]

file_polynomial = 'klein/key_equation.anf'

SB = SBox(S)
SS = [SB.to_bits(SB(s),SB.n) for s in range(len(S))]
kp = [a for a in range(24,32)] + [a for a in range(0,24)] + [a for a in range(56,64)] + [a for a in range(32,56)]
sbox = SBox(*[S])
P.<y0,y1,y2,y3,x0,x1,x2,x3> = BooleanPolynomialRing(8)
py_p = sbox.polynomials([x0,x1,x2,x3],[y0,y1,y2,y3], groebner=True)
py_s = map(lambda x: str(x), py_p)

with open(file_polynomial,"w") as file:
    file.write("ANF =\n[")
    for r in range(1,13):
        for i in range(64):
            m = str(i).zfill(2)
            n_m = str(kp[i]).zfill(2)
            file.write("kr_%s_%s"%(r,n_m) + " + ki_%s_%0s"%(r,m)+",\n")
        for i in range(32):
            m = str(i).zfill(2)
            n_m = str(i+32).zfill(2)
            file.write("kw_%s_%s"%(r,m) +  " + kr_%s_%0s"%(r,n_m)+",\n")
        for i in range(32):
            m = str(i).zfill(2)
            n_m = str(i+32).zfill(2)
            file.write("kw_%s_%s"%(r,n_m) + " + kr_%s_%0s"%(r,m) +" + kr_%s_%0s"%(r,n_m+",\n"))
        for i in range(16):
            m = str(i).zfill(2)
            file.write("ki_%s_%s"%(r+1,m) + " + kw_%s_%s"%(r,m)+",\n")

        r_bits = [(r >> i) & 1 for i in range(7,-1,-1)]

        for i in range(16,24):
            m = str(i).zfill(2)
            file.write("ki_%s_%s"%(r+1,m) + " + kw_%s_%s"%(r,m) + " + %s"%(r_bits[i-16])+",\n")
        for i in range(24,40):
            m = str(i).zfill(2)
            file.write("ki_%s_%s"%(r+1,m) + " + kw_%s_%s"%(r,m)+",\n")
        for s in range(4):
            for py in py_s:
                newp = py.replace("x0","kw_%s_%s"%(r,  40+4*s))
                newp = newp.replace("x1","kw_%s_%s"%(r,  41+4*s))
                newp = newp.replace("x2","kw_%s_%s"%(r,  42+4*s))
                newp = newp.replace("x3","kw_%s_%s"%(r,  43+4*s))
                newp = newp.replace("y0","ki_%s_%s"%(r+1,40+4*s))
                newp = newp.replace("y1","ki_%s_%s"%(r+1,41+4*s))
                newp = newp.replace("y2","ki_%s_%s"%(r+1,42+4*s))
                newp = newp.replace("y3","ki_%s_%s"%(r+1,43+4*s))
                file.write(newp+",\n")
        for i in range(56,64):
            m = str(i).zfill(2)
            file.write("ki_%s_%s"%(r+1,m) + " + kw_%s_%s"%(r,m)+",\n")
    file.write("]\n")